## Add NTD Data to Section 1

In [1]:
import _operators_prep as op_prep
import pandas as pd



In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

### NTD 
* https://dbt-docs.calitp.org/#!/model/model.calitp_warehouse.dim_annual_ntd_agency_information
* https://www.transit.dot.gov/ntd/data-product/2022-annual-database-agency-information
* 

In [3]:
def load_ntd(year: int) -> pd.DataFrame:
    df = (
        tbls.mart_ntd.dim_annual_ntd_agency_information()
        >> filter(_.year == year, _.state == "CA", _._is_current == True)
        >> select(
            _.number_of_state_counties,
            _.uza_name,
            _.density,
            _.number_of_counties_with_service,
            _.state_admin_funds_expended,
            _.service_area_sq_miles,
            _.population,
            _.service_area_pop,
            _.subrecipient_type,
            _.primary_uza,
            _.reporter_type,
            _.organization_type,
            _.agency_name,
            _.voms_pt,
            _.voms_do,
        )
        >> collect()
    )

    cols = list(df.columns)
    df2 = df.sort_values(by=cols, na_position="last")
    df3 = df2.groupby("agency_name").first().reset_index()

    return df3

###  Mobility Marketplace provider map
* https://dbt-docs.calitp.org/#!/model/model.calitp_warehouse.dim_mobility_mart_providers

In [4]:
def load_mobility()->pd.DataFrame:
    df = (
    tbls.mart_transit_database.dim_mobility_mart_providers()
     >> select(
        _.agency_name,
        _.counties_served,
        _.hq_city,
        _.hq_county,
        _.is_public_entity,
        _.is_publicly_operating,
        _.funding_sources,
        _.on_demand_vehicles_at_max_service,
        _.vehicles_at_max_service
    )
    >> collect()
    )
    
    cols = list(df.columns)
    df2 = df.sort_values(by=cols, na_position='last')
    df2 = df2.sort_values(by=["on_demand_vehicles_at_max_service","vehicles_at_max_service"], ascending = [False, False])
    df3 = df2.groupby('agency_name').first().reset_index()
    return df3

#### Merge mobility w/ NTD

In [5]:
def merge_ntd_mobility(year:int)->pd.DataFrame:
    ntd = load_ntd(year)
    mobility = load_mobility()
    m1 = pd.merge(
    mobility,
    ntd,
    how="inner",
    on="agency_name")
    agency_dict = {
    "City of Fairfield, California": "City of Fairfield",
    "Livermore / Amador Valley Transit Authority": "Livermore-Amador Valley Transit Authority",
    "Nevada County Transit Services": "Nevada County",
    "Omnitrans": "OmniTrans"}
    
    m1.agency_name = m1.agency_name.replace(agency_dict)
    return m1

#### Merge mobility w/ relevant operators

In [6]:
def ntd_operator_info(year:int)->pd.DataFrame:
    ntd_mobility_df = merge_ntd_mobility(year)
    op_profiles = op_prep.operators_with_rt()[['organization_name']]
    m1 = pd.merge(op_profiles, ntd_mobility_df,
                 how = "inner", left_on = ["organization_name"],
                 right_on = ["agency_name"])
    return m1

In [7]:
ntd_operator_info(2022)

,organization_name,agency_name,counties_served,hq_city,hq_county,is_public_entity,is_publicly_operating,funding_sources,on_demand_vehicles_at_max_service,vehicles_at_max_service,number_of_state_counties,uza_name,density,number_of_counties_with_service,state_admin_funds_expended,service_area_sq_miles,population,service_area_pop,subrecipient_type,primary_uza,reporter_type,organization_type,voms_pt,voms_do
0,Antelope Valley Transit Authority,Antelope Valley Transit Authority,Los Angeles,Lancaster,Los Angeles,True,True,5307;5310;5311,NaN,88.00,NaN,"Palmdale--Lancaster, CA",4241.00,NaN,NaN,1200.00,359559.00,349050.00,None,None,Full Reporter,Public Agency or Authority of Transit Service,88.00,NaN
1,Butte County Association of Governments,Butte County Association of Governments,Butte,Chico,Butte,True,True,5307;5310;5311;5339,NaN,36.00,NaN,"Chico, CA",3304.00,NaN,NaN,186.00,111411.00,162252.00,None,None,Full Reporter,COG or Other Planning Agency,36.00,NaN
2,Alameda-Contra Costa Transit District,Alameda-Contra Costa Transit District,Alameda,Oakland,Alameda,True,True,5307;5308;5309,399.00,546.00,NaN,"San Francisco--Oakland, CA",6943.00,NaN,NaN,364.00,3515933.00,1586454.00,None,None,Full Reporter,Public Agency or Authority of Transit Service,147.00,399.00
3,Central Contra Costa Transit Authority,Central Contra Costa Transit Authority,Alameda;Contra Costa,Concord,Contra Costa,True,True,5307;5309;5311,78.00,118.00,NaN,"Concord--Walnut Creek, CA",3064.00,NaN,NaN,274.00,538583.00,627597.00,None,None,Full Reporter,Public Agency or Authority of Transit Service,40.00,78.00
4,City of Fairfield,City of Fairfield,Solano,Fairfield,Solano,True,True,5307,NaN,33.00,NaN,"Fairfield, CA",3679.00,NaN,NaN,129.00,150122.00,505849.00,None,None,Full Reporter,County or Local Government Unit or Department of Transportation,33.00,NaN
5,Marin County Transit District,Marin County Transit District,Marin,San Rafael,Marin,True,True,5307;5310;5311;5320;5339,NaN,80.00,NaN,"San Francisco--Oakland, CA",6943.00,NaN,NaN,520.00,3515933.00,262321.00,None,None,Full Reporter,Public Agency or Authority of Transit Service,80.00,NaN
6,City and County of San Francisco,City and County of San Francisco,San Francisco,San Francisco,San Francisco,True,True,5307;5309;5310;5337;5339,658.00,752.00,NaN,"San Francisco--Oakland, CA",6943.00,NaN,NaN,49.00,3515933.00,842754.00,None,None,Full Reporter,County or Local Government Unit or Department of Transportation,94.00,658.00
7,City of Petaluma,City of Petaluma,Sonoma,Petaluma,Sonoma,True,True,5307,NaN,13.00,NaN,"Petaluma, CA",3111.00,NaN,NaN,14.00,65227.00,60520.00,None,None,Full Reporter,County or Local Government Unit or Department of Transportation,13.00,NaN
8,San Mateo County Transit District,San Mateo County Transit District,San Mateo,San Carlos,San Mateo,True,True,5303;5307;5311;5317,157.00,312.00,NaN,"San Francisco--Oakland, CA",6943.00,NaN,NaN,97.00,3515933.00,765551.00,None,None,Full Reporter,Public Agency or Authority of Transit Service,155.00,157.00
9,Santa Clara Valley Transportation Authority,Santa Clara Valley Transportation Authority,Santa Clara,San Jose,Santa Clara,True,True,5307;5309;5311;5337,363.00,496.00,NaN,"San Jose, CA",6436.00,NaN,NaN,346.00,1837446.00,1894783.00,None,None,Full Reporter,Public Agency or Authority of Transit Service,133.00,363.00


### Dim Organizations
* https://dbt-docs.calitp.org/#!/model/model.calitp_warehouse.dim_organizations

In [8]:
# rganizations = tbls.mart_transit_database.dim_organizations() >> collect()

In [9]:
# organizations.head(2)